<a href="https://colab.research.google.com/github/athenakle/Bnfo301_Le_Athena/blob/main/BNFO301_2023_LabExtraCredit_Athena%20Le.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>BNFO301 ExtraCredit Homework Assignment </h1>

In this assignment, has 4 problems, worth 5 points each (20 points total).  The lecture slides discribe the basics of fastq format, sequencing technology, and read assembly.





### Setup
Read the Sequence Data File.  Leave this as is and run.

In [29]:
import os.path
# Load the genbank file 
DATA_FILE_GITHUB = "https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/SampleFqfile_301.fq"
DEFAULT_FILE_NAME = 'SampleFqfile_301.fq'
fileName = DEFAULT_FILE_NAME
#Does the file exists locally, if not get it from the github
if not os.path.exists(fileName):
  #Load the file from Github to the local folder
  !wget --no-check-certificate --content-disposition $DATA_FILE_GITHUB
   
print("Reading file:", fileName)


Reading file: SampleFqfile_301.fq


###Problem 1

Read in the data, return header, squences, and scores<br>



Problem 1.a) Fastq files contain associated sets of data on multiple lines.  In this problem, you need to create a class to represent a sequence, it's header, and it's score.

In [30]:
class Sequences:
    def __init__(self, header, seq, score=None):
        self.header = header
        self.seq = seq
        self.score = score

    def __str__(self):
        s = f"{self.header}\n{self.seq}\n"
        if self.score:
            s += f"{self.score}\n"
        return s


Problem 1.b) Create a definition to read the FASTQ file and return a list containing the header, DNA_sequence, and Phred_scores.  Recall that ASCII encoded scores include "@" and "+".  Unlike reading in fasta formated files, we simply state that the line begins with ">".  I find it is useful to create a flag.  This allows me to be informed about line order.

Here is an example, the Last List Object:

"Header:@A00835:187:HKWNMDSXY:1:1101:29008:1000 1:N:0:CCGCGGTT+CTAGCGCT

Sequence:CGGAAAGTTGGGTAGGGGGAGGGAGGGAGTGACATGGTTTTTTAAAAGCAAATTTTCCTTTTTGGGTTGAGCAAACCTCGTTACGTTGTGTGTGTTTTGTTTCTTTGTGTGGCGGGTGTCAGGCTTGCTTAATTTTGAACCGCTCAGTAT

Score:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF:FFFFFFFFF"

In [31]:
def LoadFastq(myfile):
    HEADERStartChar = "@"
    ScoreSeparator = "+"
    fh = open(myfile, "r")
    data = fh.readlines()
    fh.close()
    sequenceList = []
    header = ""
    sequence = ""
    score = ""
    flag = 0
    for line in data:
        if line[0] == HEADERStartChar:
            if flag == 0:
                header = line.strip()
                flag = 1
            else:
                sequenceList.append(Sequences(header, sequence, score))
                header = line.strip()
                sequence = ""
                score = ""
        elif line[0] == ScoreSeparator:
            flag = 2
        else:
            if flag == 1:
                sequence += line.strip()
            elif flag == 2:
                score += line.strip()
    sequenceList.append(Sequences(header, sequence, score))
    return sequenceList


### Problem 2
Create a definition to return the "Quality Score."  The data provided to you in the fastq file, repersents short sequence reads generated using Illumina HiSeq technology.  Illumina encodes the Quality score for each base using the ASCII character + 33.  You can use the ord function (for example "ord(A)" should return 65) to find the value associated with a character and substract 33 to get the Phred score (therefore, A = 32). <br>

Here is a list of characters and their score: https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm

In [ ]:
def findQScore(c):
    # Calculate the ASCII code for the input character
    ascii_code = ord(c)
    # Subtract the ASCII code for the letter '!' to get the Phred+33 quality score
    q_score = ascii_code - 33
    # Return the quality score as an integer
    return q_score


###  Problem 3

Soft-trimming:  Create a defintion that will remove bases from the 3' end of reads based on their associated Quality score.  Starting with the 3' base, remove any base with a score below 28.  Use a sliding window of 1 base and stop trimming when you reach a base with a score > 28.  You only need to trim the sequence line itself.  You do not need to trim the "Quality line".

In [36]:
def softTrim(seqObject, compareScore=28):
    # Get the sequence from the input sequence object
    seq = seqObject.seq
    # Initialize the trim point to None
    trim_point = None
    # Loop over the sequence in reverse, starting from the second-to-last base
    for i in range(len(seq)-2, -1, -1):
        # Get the quality score for the current base and the next base
        q_score_current = findQScore(seq[i])
        q_score_next = findQScore(seq[i+1])
        # Check if the quality score of the current base is less than the specified threshold
        if q_score_current < compareScore:
            # If it is, set the trim point to the current index and break out of the loop
            trim_point = i
            break
        # Check if the quality score of the next base is greater than the specified threshold
        if q_score_next > compareScore:
            # If it is, set the trim point to the current index and break out of the loop
            trim_point = i+1
            break
    # Check if a trim point was found
    if trim_point is None:
        # If not, return the original sequence
        return seq
    else:
        # If a trim point was found, truncate the sequence at the trim point and return the truncated sequence
        truncated_seq = seq[:trim_point]
        return truncated_seq


### Problem 4
#### Bring it all together and use the defition created in probelm 3 to soft trim all the sequence in "seqList"  Print the each list element followed by the trimmed sequence.  You do not need to trim the "Quality line" itself.


In [34]:
i = 1
for seq in seqList:
    # Print the original sequence
    print(f"\nSequence {i}:")
    print(seq)
    
    # Do the soft trimming using the definition from problem 3
    trimmed_seq = soft_trim(seq.seq, seq.letter_annotations["phred_quality"])
    
    # Print the trimmed sequence
    print(f"Trimmed Sequence {i}:")
    print(trimmed_seq)
    
    i += 1


NameError: ignored